In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import copy
import math
import spacy
import os

In [2]:
class Transformer(nn.Module):
    """
    A standard encoder-decoder architecture. Acts as a base for this and other models.
    The model takes in encoder, decoder, src embed, target embed objects.
    And finally a generator object that produces predictions from the decoder.
    """

    def __init__(self, encoder, decoder, src_embed, tgt_embed, generator):
        super(Transformer, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.src_embed = src_embed
        self.tgt_embed = tgt_embed
        self.generator = generator

    def forward(self, src, src_mask, tgt, tgt_mask):
        return self.decode(self.encode(src, src_mask), src_mask, tgt, tgt_mask)
    
    def encode(self, src, src_mask):
        return self.encoder(self.src_embed(src), src_mask)
    
    def decode(self, memory, src_mask, tgt, tgt_mask):
        return self.decoder(self.tgt_embed(tgt), memory, src_mask, tgt_mask)

In [3]:
class Generator(nn.Module):
    """Final generation step with linear layer and softmax for output probs"""

    def __init__(self, d_model, vocab_size):
        super(Generator, self).__init__()
        self.linear_layer = nn.Linear(d_model, vocab_size)

    def forward(self, x):
        return F.log_softmax(self.linear_layer(x), dim=-1)

In [4]:
class LayerNorm(nn.Module):
    def __init__(self, features, eps=1e-6):
        super(LayerNorm, self).__init__()
        self.a_2 = nn.Parameter(torch.ones(features))
        self.b_2 = nn.Parameter(torch.zeros(features))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.a_2 * (x-mean) / (std + self.eps) + self.b_2

In [5]:
# Transformer consists of stacked layers of Encoder and Decoder


def generate_clones(module, N):
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

# Encoder stack
class EncoderStack(nn.Module):
    """
    Takes in single encoder layer. Produce stacked layers and passes input thru each layer
    """
    def __init__(self, encoder_layer, N=6):
        super(EncoderStack, self).__init__()
        self.layers = generate_clones(encoder_layer, N)
        self.norm = LayerNorm(encoder_layer.size)

    def forward(self, x, src_mask):
        """
        Pass input thru each layer
        """
        for layer in self.layers:
            x = layer(x, src_mask)
        return self.norm(x)

In [6]:
class LayerNorm(nn.Module):
    def __init__(self, features, eps=1e-6):
        super(LayerNorm, self).__init__()
        self.a_2 = nn.Parameter(torch.ones(features))
        self.b_2 = nn.Parameter(torch.zeros(features))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.a_2 * (x-mean) / (std + self.eps) + self.b_2

In [7]:
class SublayerConnection(nn.Module):
    def __init__(self, size, dropout=0.1):
        super(SublayerConnection, self).__init__()
        self.norm = LayerNorm(size)
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, x, sublayer):
        """Apply residual connection to the output of sublayer and layer norm the result"""
        return self.norm(x + sublayer(x))

In [8]:
# A single encoder layer. It consists of two sub layers. The first is a multi-head self-attention module and 
# a fully connected position-wise feed forward network

class EncoderLayer(nn.Module):
    def __init__(self, model_size, self_attn, feed_forward, dropout=0.1):
        super(EncoderLayer, self).__init__()
        self.self_attn = self_attn
        self.feed_forward = feed_forward
        self.size = model_size
        self.sublayers = generate_clones(SublayerConnection(model_size, dropout), 2)
    
    def forward(self, x, mask):
        x = self.sublayers[0](x, lambda x: self.self_attn(x, x, x, mask))
        return self.sublayers[1](x, self.feed_forward)


In [9]:
class DecoderStack(nn.Module):
    def __init__(self, decoder_layer, N=6):
        super(DecoderStack, self).__init__()
        self.layers = generate_clones(decoder_layer, N)
        self.norm = LayerNorm(decoder_layer.size)
    
    def forward(self, x, memory, src_mask, tgt_mask):
        for layer in self.layers:
            x = layer(x, memory, src_mask, tgt_mask)
        return self.norm(x)

In [10]:
class DecoderLayer(nn.Module):
    """
    Decoder layer has 3 sublayers: self-attention, source attention and feed forward
    """
    def __init__(self, model_size, self_attn, src_attn, feed_forward, dropout):
        super(DecoderLayer, self).__init__()
        self.size = model_size
        self.self_attn = self_attn
        # source attention module
        self.src_attn = src_attn
        self.feed_forward = feed_forward
        self.dropout = nn.Dropout(dropout)
        self.sublayers = generate_clones(SublayerConnection(model_size, dropout), 3)

    def forward(self, x, memory, src_mask, tgt_mask):
        m = memory
        # pass the data thru self attention module
        x = self.sublayers[0](x, lambda x: self.self_attn(x, x, x, tgt_mask))
        # pass the data thru source attention module
        x = self.sublayers[1](x, lambda x: self.src_attn(x, m, m, src_mask))
        # pass the data thru feed forward module
        return self.sublayers[2](x, self.feed_forward)

In [11]:
# Attention function
# Attention(Q, K, V) = softmax(Q @ K.T / sqrt(d_k)) @ V
def attention(query, key, value, mask=None, dropout=None):
    d_k = query.size(-1)
    scores = torch.matmul(query, key.transpose(-1, -2)) / math.sqrt(d_k)

    if mask is not None:
        scores = scores.masked_fill(mask == 0, -1e9)
        print()#do something

    attn_scores = F.softmax(scores, dim=-1)
    if dropout is not None:
        attn_scores = dropout(attn_scores)
    result = torch.matmul(attn_scores, value)
    return result, attn_scores

In [12]:
# Multi head Attention

class MultiHeadAttention(nn.Module):
    def __init__(self, model_size, heads, dropout=0.1):
        super(MultiHeadAttention, self).__init__()
        self.model_size = model_size
        self.heads = heads
        self.dropout = nn.Dropout(dropout)
        print("model_size", model_size)
        print("heads", heads)
        assert model_size % heads == 0, "model_size must be divisible by heads"
        self.d_k = model_size // heads
        self.linear_layers = generate_clones(nn.Linear(model_size, model_size), 4)
        self.attn = None

    def forward(self, query, key, value, mask=None):
        if mask is not None:
            mask = mask.unsqueeze(1)
        n_batches = query.size(0)

        # linearly project query, key and value in to number of heads times
        print(query.shape)
        print(key.shape)
        print(value.shape)
        query, key, value = [
            layer(x).view(n_batches, -1, self.heads, self.d_k).transpose(1, 2) 
            for layer, x in zip(self.linear_layers, (query, key, value))
        ]

        # calculate attention using scaled dot product
        x, self.attn = attention(query, key, value, mask=mask, dropout=self.dropout)

        # x shape would be [n_batches, heads, seq_len, d_k]
        # concat all the heads and apply final linear layer
        x = x.transpose(1, 2).contiguous().view(n_batches, -1, self.heads * self.d_k)
        del query
        del key
        del value
        return self.linear_layers[-1](x)



In [13]:
# position wise feed forward network

class PositionwiseFeedForward(nn.Module):
    def __init__(self, model_size, d_ff, dropout=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.layer1 = nn.Linear(model_size, d_ff)
        self.layer2 = nn.Linear(d_ff, model_size)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x):
        return self.layer2(self.dropout(self.layer1(x).relu()))

In [14]:
# Embeddings
class Embeddings(nn.Module):
    def __init__(self, d_model, vocab):
        super(Embeddings, self).__init__()
        self.lut = nn.Embedding(vocab, d_model)
        self.d_model = d_model

    def forward(self, x):
        return self.lut(x) * math.sqrt(self.d_model)

In [15]:
# Positional Encoding

# PE helps model to understand the order of sequence of words. Since model 
# doesn't have any recurrence or convolution layers

class PositionalEncoding(nn.Module):
    "Implement the PE function."

    def __init__(self, d_model, dropout, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(
            torch.arange(0, d_model, 2) * -(math.log(10000.0) / d_model)
        )
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer("pe", pe)

    def forward(self, x):
        x = x + self.pe[:, : x.size(1)].requires_grad_(False)
        return self.dropout(x)

In [16]:
def make_model(src_vocab, tgt_vocab, N=6, model_size=512, d_ff=2048, h=8, dropout=0.1):
    c = copy.deepcopy
    attn = MultiHeadAttention(model_size, h, dropout)
    ffn = PositionwiseFeedForward(model_size, d_ff, dropout)
    position = PositionalEncoding(model_size, dropout)
    model = Transformer(
        EncoderStack(EncoderLayer(model_size, c(attn), c(ffn), dropout), N),
        DecoderStack(DecoderLayer(model_size, c(attn), c(attn), c(ffn), dropout), N),
        nn.Sequential(Embeddings(model_size, src_vocab), c(position)),
        nn.Sequential(Embeddings(model_size, tgt_vocab), c(position)),
        Generator(model_size, tgt_vocab)
    )

    # This was important from their code.
    # Initialize parameters with Glorot / fan_avg.
    for p in model.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)
    return model

In [17]:
def subsequent_mask(size):
    "Mask out subsequent positions."
    attn_shape = (1, size, size)
    subsequent_mask = torch.triu(torch.ones(attn_shape), diagonal=1).type(
        torch.uint8
    )
    return subsequent_mask == 0

In [18]:
def inference_test():
    test_model = make_model(11, 11, 2)
    src = torch.LongTensor([[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]])
    src_mask = torch.ones(1, 1, 10)

    m = test_model.encode(src, src_mask)
    print(m.shape)

    ys = torch.zeros(1, 1).type_as(src)

    out = test_model.decode(
            m, src_mask, ys, subsequent_mask(ys.size(1)).type_as(src.data)
        )
    prob = test_model.generator(out[:, -1])
    print(prob.shape)

inference_test()

model_size 512
heads 8
torch.Size([1, 10, 512])
torch.Size([1, 10, 512])
torch.Size([1, 10, 512])

torch.Size([1, 10, 512])
torch.Size([1, 10, 512])
torch.Size([1, 10, 512])

torch.Size([1, 10, 512])
torch.Size([1, 1, 512])
torch.Size([1, 1, 512])
torch.Size([1, 1, 512])

torch.Size([1, 1, 512])
torch.Size([1, 10, 512])
torch.Size([1, 10, 512])

torch.Size([1, 1, 512])
torch.Size([1, 1, 512])
torch.Size([1, 1, 512])

torch.Size([1, 1, 512])
torch.Size([1, 10, 512])
torch.Size([1, 10, 512])

torch.Size([1, 11])


In [19]:
# Model Training 

# Batch object and masking
class Batch:
    def __init__(self, src, tgt=None, pad=2):
        self.src = src
        # hide padding tokens/words. value with 2 gets masked
        self.src_mask = (src != pad).unsqueeze(-2)
        if tgt is not None:
            self.tgt = tgt[:, :-1]
            self.tgt_y = tgt[:, 1:]
            self.tgt_mask = self.make_std_mask(self.tgt, pad)

    @staticmethod
    def make_std_mask(tgt, pad):
        "create mask to hide padding and future words"
        # hide padding words
        tgt_mask = (tgt!= pad).unsqueeze(-2)
        tgt_mask = tgt_mask & subsequent_mask(tgt.size(-1)).type_as(
            tgt_mask.data
        )
        return tgt_mask


In [20]:
# Tokenization

# we are considering multi30k German-English translation task. The paper
# used WMT task which is much bigger
def load_tokenizers():
    try:
        spacy_de = spacy.load('de_core_news_sm')
    except IOError:
        os.system(f'python -m spacy download de_core_news_sm')
        spacy_de = spacy.load('de_core_news_sm')

    try: 
        spacy_en = spacy.load('en_core_web_sm')
    except IOError:
        os.system(f'python -m spacy download en_core_web_sm')
        spacy_en = spacy.load('en_core_web_sm')

    return spacy_de, spacy_en

In [21]:
spacy_de, spacy_en = load_tokenizers()

In [22]:
# build vocabulary
import torchtext.datasets as datasets
from torchtext.vocab import build_vocab_from_iterator


def yield_tokens(data_iter, tokenizer, index):
    for sentence_tuple in data_iter:
        yield tokenizer(sentence_tuple[index])

def tokenize(text, tokenizer):
    return [tok.text for tok in tokenizer.tokenizer(text)]

def build_vocabulary(spacy_de, spacy_en):
    """
    Takes in respective language tokenizers and return source and target vocab objects
    """

    def tokenize_de(text):
        return tokenize(text, spacy_de)
    
    def tokenize_en(text):
        return tokenize(text, spacy_en)
    
    print('Building German vocabulary')
    train, val, _ = datasets.Multi30k(language_pair=('de', 'en'))
    vocab_src = build_vocab_from_iterator(
        yield_tokens(train+val, tokenize_de, 0),
        min_freq=2,
        specials=["<s>", "</s>", "<blank>", "<unk>"]
    )

    print('Building english vocabulary')
    train, val, _ = datasets.Multi30k(language_pair=('de', 'en'))

    vocab_tgt = build_vocab_from_iterator(
        yield_tokens(train+val, tokenize_en, 1),
        min_freq=2,
        specials=["<s>", "</s>", "<blank>", "<unk>"]
    )

    vocab_src.set_default_index(vocab_src["<unk>"])
    vocab_tgt.set_default_index(vocab_tgt["<unk>"]) 

    return vocab_src, vocab_tgt

In [23]:
from os.path import exists

def load_vocab(spacy_de, spacy_en):
    if not exists("vocab.pt"):
        vocab_src, vocab_tgt = build_vocabulary(spacy_de, spacy_en)
        torch.save((vocab_src, vocab_tgt), "vocab.pt")
    else:
        vocab_src, vocab_tgt = torch.load("vocab.pt")
    print("Finished.\nVocabulary sizes:")
    print(len(vocab_src))
    print(len(vocab_tgt))
    return vocab_src, vocab_tgt


vocab_src, vocab_tgt = load_vocab(spacy_de, spacy_en)

Building German vocabulary
ShardingFilterIterDataPipe
ShardingFilterIterDataPipe
Building english vocabulary
Finished.
Vocabulary sizes:
8185
6291


In [25]:
from torch.nn.functional import pad

def collate_batch(
    batch,
    src_tokenizer, 
    tgt_tokenizer,
    vocab_src, 
    vocab_tgt,
    device,
    max_padding=128,
    pad_id=2
):
    bs_id = torch.tensor([0], device=device) # <s> token id
    eos_id = torch.tensor([1], device=device) # </s> token id
    src_list = []
    tgt_list = []
    for (src, tgt) in batch:
        processed_src = torch.cat(
            [
                bs_id,
                torch.tensor(
                    # tokenize src sentence and get list of ids from vocab_src object
                    vocab_src(src_tokenizer(src)),
                    dtype=torch.int64,
                    device=device
                ),
                eos_id,
            ],
            0,
        )
        processed_tgt = torch.cat(
            [
                bs_id,
                torch.tensor(
                    # tokenize src sentence and get list of ids from vocab_src object
                    vocab_tgt(tgt_tokenizer(tgt)),
                    dtype=torch.int64,
                    device=device
                ),
                eos_id,
            ],
            0,
        )

        src_list.append(
            pad(
                processed_src,
                (0, max_padding - len(processed_src)),
                value=pad_id
            )
        )

        tgt_list.append(
            pad(
                processed_tgt,
                (0, max_padding - len(processed_tgt)),
                value=pad_id
            )
        )

    return (torch.stack(src_list), torch.stack(tgt_list))


In [ ]:
# DataLoaders
from torchtext.data.functional import to_map_style_dataset
from torch.utils.data import DataLoader

def create_dataloaders(
    device,
    vocab_src,
    vocab_tgt,
    spacy_de,
    spacy_en,
    max_padding=128,
    batch_size=12000,
    is_distributed=True
):
    def tokenize_de(text):
        return tokenize(text, spacy_de)
    
    def tokenize_en(text):
        return tokenize(text, spacy_en)

    def collate_fn(batch):
        return collate_batch(
            batch,
            tokenize_de,
            tokenize_en,
            vocab_src,
            vocab_tgt,
            device,
            max_padding=max_padding,
            pad_id=vocab_src.get_stoi()['<blank>']
        )

    train_iter, valid_iter, _ = datasets.Multi30k(language_pair=('de', 'en'))

    train_iter_map = to_map_style_dataset(train_iter)

    valid_iter_map = to_map_style_dataset(valid_iter)

    train_dataloader = DataLoader(
        train_iter_map,
        batch_size=batch_size,
        shuffle=True,
        collate_fn=collate_fn,
    )

    valid_dataloader = DataLoader(
        valid_iter_map,
        batch_size=batch_size,
        shuffle=True,
        collate_fn=collate_fn,
    )

    return train_dataloader, valid_dataloader